In [1]:
import pandas as pd
import numpy as np

In [34]:
# initialize data from based on some column list
my_columns = ["a", "b", "c"]
pd.DataFrame(columns=my_columns)

,a,b,c


In [12]:
# JOIN/MERGE two tables

curvedata = pd.DataFrame({
    'ID': [1, 1, 1, 2, 2, 5, 5],
    'X': [0, 10, 19, 21, 22, 50, 55],
    'Y': [0, 100, 190, 210, 220, 500, 550]
})

tank = pd.DataFrame({
    'ID': [1, 2, 3],
    'Name': ['A', 'B', 'C'],
    'Dia':[2,3, None],
    'Min':[0,1,2],
    'Max':[10,11,12],
    'Curve':[None, 2, 1]
})

xl_tank = pd.DataFrame({
    'ID': [1,1,2]
})


merged_tank = pd.merge(xl_tank, tank, on=['ID'], how='inner')
merged_tank
#merged_tank[merged_tank.Dia==2]




,ID,Name,Dia,Min,Max,Curve
0,1,A,2.0,0,10,NaN
1,1,A,2.0,0,10,NaN
2,2,B,3.0,1,11,2.0


In [57]:
# Query DataFrame
tank.query('Name=="C" or Name=="B"')


,ID,Name,Dia,Min,Max,Curve
1,2,B,3.0,1,11,2.0
2,3,C,NaN,2,12,1.0


In [35]:

xl_tank_2 = pd.DataFrame({
    'ID': [3, 4]
})

xl_other = pd.DataFrame({
    'Someting': [0,0]
})


# Append/Concat tables
cdf = pd.concat([xl_tank_2, xl_other], sort=False)


In [27]:
# INSERT AT given location
xl_tank.loc[-1, 'ID'] ="[some text]"  # add a row
xl_tank.index = xl_tank.index + 1  # shifting index
xl_tank = xl_tank.sort_index()  # sorting by index

xl_tank

,ID
0,[some text]
1,1
2,2


In [5]:
# FIND DUPLICATES
tableA = pd.DataFrame({
    'ID': [1, 2, 3, 4, 4, 1],
    'Name':["a", "b", "c", "d", "e", "f"],
    'Comments': ["", "", "", "", "hi", ""]
})

tableA["ID"].is_unique  # > False

tableA["ID"].drop_duplicates() # > Gives only the unique values

tableA["Dups"] = tableA["ID"].duplicated(keep='first') # > Add a column with True/False as values

just_dups = tableA[(tableA["Dups"]==True)]  # > Filter the table to just the duplicate values

# append some comments on duplicate values
tableA["Comments"] = just_dups[["Comments"]].apply(lambda x: x + "|Found duplicates", axis=1)

# Remove the Dups colmn
tableA.drop('Dups', axis=1, inplace=True) #axis: 1=Col, 0=Row

tableA

,ID,Name,Comments
0,1,a,NaN
1,2,b,NaN
2,3,c,NaN
3,4,d,NaN
4,4,e,hi|Found duplicates
5,1,f,|Found duplicates


In [15]:
# use of MASK to find Duplicates
tableA = pd.DataFrame({
    'ID': [1, 2, 3, 4, 4, 1],
    'Name':["a", "b", "c", "d", "e", "f"],
    'Comments': ["", "", "", "", "hi", ""]
})

has_duplicate = tableA["ID"].duplicated(keep='first') # > Add a column with True/False as values
tableA.loc[has_duplicate, 'Comments'] += "|Is a Duplidate"  # > Update table based on the filter statement
tableA

,ID,Name,Comments
0,1,a,
1,2,b,
2,3,c,
3,4,d,
4,4,e,hi|Is a Duplidate
5,1,f,|Is a Duplidate


In [12]:
# Double MASKING
tableA["Dups"] = tableA["ID"].duplicated(keep='first') # > Add a column with True/False as values

mask_dups = tableA["Dups"] == True # > Create a filter statement ~ => dfA[(df.ID > 3) & (df.ID < 5)]
mask_name = tableA["Name"] == 'e'
mask = mask_dups & mask_name
tableA[mask]



,ID,Name,Comments,Dups
4,4,e,hi|Is a Duplidate,True


In [17]:
# MASK on LIST
name = ["a", "e"]
name_bit_or = "|".join(map(str, name)
mask = tableA["Name"].astype(str).str.contains(name_bit_or)
tableA[mask]

,ID,Name,Comments
0,1,a,
4,4,e,hi|Is a Duplidate


In [66]:
# Group By based on another function
p_ids = pd.DataFrame({
    'ID': [1,1,1,2,2,3,3,3],
    'SEQ': [0,1,2,0,1,0,1,2],
    'X': [10,11,12,20,21,30,31,32],
    'Y': [100,110,120,200,210,300,310,320]
})

p_ids.sort_values(['ID','SEQ'], inplace=True)

ids = [1,3]
ids = "|".join(map(str, ids))
mask = p_ids['ID'].astype(str).str.contains(ids)
p_ids = p_ids[mask]


def get_average(data):
    #data["SEQ"] = data["SEQ"].sum()
    data["X"] = data["X"].mean()
    data["Y"] = data["Y"].mean()
    return data

p_ids = p_ids.groupby('ID').apply(get_average).drop_duplicates("ID", keep='first')
p_ids




,ID,SEQ,X,Y
0,1,0,11.0,110.0
5,3,0,31.0,310.0


In [7]:
# Group By
p_ids = pd.DataFrame({
    'ID': [1,1,1,2,2,3,3,3],
    'SEQ': [0,1,2,0,1,0,1,2],
    'X': [10,11,12,20,21,30,31,32],
    'Y': [100,110,120,200,210,300,310,320]
})

#grouped = p_ids.groupby(["ID"]).agg({'X': ['sum', 'count']})
grouped = p_ids.groupby(["ID"]).agg(['sum', 'count'])
grouped


SEQ         X          Y      
   sum count sum count  sum count
ID                               
1    3     3  33     3  330     3
2    1     2  41     2  410     2
3    3     3  93     3  930     3

In [8]:
# Group By Specific Column
p_ids = pd.DataFrame({
    'ID': [1,1,1,2,2,3,3,3],
    'SEQ': [0,1,2,0,1,0,1,2],
    'X': [10,11,12,20,21,30,31,32],
    'Y': [100,110,120,200,210,300,310,320]
})

grouped = p_ids.groupby(["ID"]).agg({'X': ['sum', 'count']})
grouped

X      
   sum count
ID          
1   33     3
2   41     2
3   93     3

In [33]:
# Rename Columns of a dataframe
p_ids = pd.DataFrame({
    'ID': [1,1,1,2,2,3,3,3],
    'SEQ': [0,1,2,0,1,0,1,2],
    'X': [10,11,12,20,21,30,31,32],
    'Y': [100,110,120,200,210,300,310,320]
})

p_ids.rename(index=str, columns={"ID": "EyeD", "X": "Ex"})

,EyeD,SEQ,Ex,Y
0,1,0,10,100
1,1,1,11,110
2,1,2,12,120
3,2,0,20,200
4,2,1,21,210
5,3,0,30,300
6,3,1,31,310
7,3,2,32,320


In [2]:
# Update a datafrme based on another dataframe
one = pd.DataFrame({
    "ID": [1, 2, 3, np.nan,7], 
    "B": [4, 0, np.nan, 9,70],
    "C": [7, 8, 9, 10,700]
})

two = pd.DataFrame({
    "ID": [3, 6, 7, 4], 
    "B": [4, 5, 6, 7]
})

one.set_index("ID", inplace=True)
two.set_index("ID", inplace=True)

one.update(two)
one



,B,C
ID,,
1.0,4.0,7
2.0,0.0,8
3.0,4.0,9
NaN,9.0,10
7.0,6.0,700


In [16]:
users = pd.DataFrame({
    "ID": [3, 6, 7, 4], 
    "fn": ['Aks', 'Aav', 'Ava', 'Bva'],
    "ln": ['lnAks', 'lnAav', 'lnAva', 'lnBva']
})

def get_full_name(id):
    mask = users['ID'] == id
    masked_user = users.loc[mask]
    return '%s %s' % (masked_user['fn'], masked_user['ln'])

merged = pd.merge(two, users, on='ID')
mapped = users['ID'].map(lambda x: get_full_name(x))



print(mapped)
                      

0    0    Aks\nName: fn, dtype: object 0    lnAks\n...
1    1    Aav\nName: fn, dtype: object 1    lnAav\n...
2    2    Ava\nName: fn, dtype: object 2    lnAva\n...
3    3    Bva\nName: fn, dtype: object 3    lnBva\n...
Name: ID, dtype: object


In [20]:
# FIND DUPLICATES
tableA = pd.DataFrame({
    'ID': [1, 2, 3, 4, 4, 1],
    'Name':["a", "b", "c", "d", "e", "f"],
    'Comments': ["", "", "", "", "hi", ""]
})

tableA["ID"].is_unique  # > False

tableA.drop_duplicates(["ID"], inplace=True) # > Gives only the unique values
tableA

,ID,Name,Comments
0,1,a,
1,2,b,
2,3,c,
3,4,d,


In [92]:
txt_file_path = r'D:\Office\Temp\susana\test.TXT'
txt_file_path = r'D:\Office\Temp\susana\CONSUMPTION_2017.TXT'
out_file_path = r'D:\Office\Temp\susana\summed.csv'
out_file_path_dirty = r'D:\Office\Temp\susana\removed.csv'
out_file_path_clean = r'D:\Office\Temp\susana\cleaned.csv'

df = pd.read_csv(txt_file_path, sep=r'\s+')
df["custID"] = df["aID"]


print('Source:')
print(df.shape)
print(df.columns)

mask = df["gal"].astype(str).str.contains("-") | df["ccf"].astype(str).str.contains("E") | df["add"].astype(str).str.contains("E") 
df_dirty = df[mask]
df_clean =  df[~mask]


#print(df_clean)

df_dirty.to_csv(out_file_path_dirty, index=False)
df_clean.to_csv(out_file_path_clean, index=False)

df_clean = pd.read_csv(out_file_path_clean)

print('\nclean: ')
print(df_clean.shape)

print('\ndirty: ')
print(df_dirty.shape)

df_summary = df_clean.groupby(["custID"]).agg({'ccf': ['sum', 'count']})
df_summary.to_csv(out_file_path)

print('\nsummary: ')
print(df_summary.shape)
df_summary.head()


Source:
(1653742, 8)
Index(['aID', 'sID', 'e', 'add', 'ccf', 'gal', 'unit', 'custID'], dtype='object')

clean: 
(1569747, 8)

dirty: 
(83995, 8)

summary: 
(379881, 2)


ccf      
                                          sum count
custID                                             
6000000200   12379.4022477.4019859.4027227.20     4
6000000201   40309.7239509.3645209.1238222.80     4
6000000203   21991.2023075.8042000.2045740.20     4
6000000204  50430.1665637.0074702.76101690.60     4
6000000205   41940.3647049.2054125.2865928.72     4

In [88]:
out_file_path_clean = r'D:\Office\Temp\susana\cleaned.csv'
out_file_path = r'D:\Office\Temp\susana\summed.csv'

df_clean = pd.read_csv(out_file_path_clean)

print('\nclean: ')
print(df_clean.shape)

df_clean.dtypes

# df_summary = df_clean.groupby(["custID"]).agg({'add': ['sum', 'count']})
# df_summary.to_csv(out_file_path)

# print('\nsummary: ')
# print(df_summary.shape)

# df_summary.head()


clean: 
(99999, 8)


aID         int64
sID        object
e           int64
add        object
ccf        object
gal        object
unit      float64
custID      int64
dtype: object

In [9]:
# Test replacing blanks/null to empty string''
dataB = pd.DataFrame({
    'one':[1, np.nan, 3],
    'two':[11, np.nan, 33]
})

# Print dataB
print(dataB)

dataB.replace(np.nan, '', regex=True, inplace=True)

print(dataB)


   one   two
0  1.0  11.0
1  NaN   NaN
2  3.0  33.0
  one two
0   1  11
1        
2   3  33
